# Discrete Fourier Transform

## Part 1

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

print("Input Sine Wave Signal:")
N = 360 # degrees (Number of samples)
a = random.randint(1, 100)
f = random.randint(1, 50)
p = random.randint(0, 360)
print("frequency = {:3d}".format(f))
print("amplitude = {:3d}".format(a))
print("phase ang = {:3d}".format(p))

### Manually calculate the inverse Fourier transform

We can calculate the sine wave with $A\sin(ft + \phi)$. Because we are working in degrees, and Python's `sin()` function only understands radians, we need to convert using the `radians()` function.

In [ ]:
from math import sin, pi, radians

f_list = []
for n in range(N):
    sample = a * sin(radians(f * n + p))
    f_list.append(sample)

In [ ]:
plt.plot(f_list);

We're going to implement the Fourier transform as given on Wolfram Alpha:
$$
F_n = \sum_{k=0}^{N-1} f_k e^{-2\pi ink/N}.
$$

In [ ]:
from numpy import exp

#function to calculate the Discrete Fourier Transform
def DFT(f_list):
    N = len(f_list)
    DFT_list = []
    for n in range(N):
        Fn = 0.0
        for k in range(N):
            Fn += f_list[k] * exp(-2j * pi * n * k / N)
        DFT_list.append(Fn)
    return DFT_list        

In [ ]:
DFT_list = DFT(f_list)

In [ ]:
for n, coefficient in enumerate(DFT_list[:10]):
    print("F_{:<3d} = {}".format(n, coefficient))

In [ ]:
plt.plot(np.absolute(DFT_list));

We're going to implement the *inverse* Fourier transform as given on Wolfram Alpha:
$$
f_k = \frac{1}{N} \sum_{n=0}^{N-1} F_n e^{2\pi ikn/N}.
$$

In [ ]:
#function to calculate the inverse Fourier transform
def inverse_DFT(DFT_list):
    N = len(DFT_list)
    f_list = []
    for k in range(N):
        fk = 0.0
        for n in range(N):
            fk += DFT_list[n] * exp(2j * pi * k * n / N)
        f_list.append(fk / N)
    return f_list        

In [ ]:
f_list2 = inverse_DFT(DFT_list)

In [ ]:
for n, (coefficient1, coefficient2) in enumerate(zip(f_list, f_list2[:10])):
    print("f_{:<3d} = {}\t{}".format(n, coefficient1, coefficient2))

Unfortunately, due to rounding errors, we're still seeing small imaginary values. They really are small. Let's calculate the largest absolute value of the imaginary parts:

In [ ]:
np.max(np.abs(np.imag(f_list2)))

So, indeed, rounding errors. Let's make `f_list2` real by ignoring the imaginary part:

In [ ]:
f_list2 = [np.real(u) for u in f_list2]

In [ ]:
for n, (coefficient1, coefficient2) in enumerate(zip(f_list, f_list2[:10])):
    print("f_{:<3d} = {}\t{}".format(n, coefficient1, coefficient2))

Let's see if the original and recreated values are the same:

In [ ]:
plt.plot(f_list)
plt.plot(f_list2);

They are! The plotted lines coincide.

### Again, but now with some NumPy magic

Calculating the original samples:

In [ ]:
n = np.arange(N)
f_n = a * np.sin(np.radians(f * n + p))

In [ ]:
plt.plot(f_n);

Again, the Fourier transform as given on Wolfram Alpha:
$$
F_n = \sum_{k=0}^{N-1} f_k e^{-2\pi ink/N}.
$$
First, building values for $k$, and creating $f_k$, which is simply identical to $f_n$ (we only do this to be able to write down the python code as closely as possible to the mathematical formula):

In [ ]:
k = np.arange(N)
f_k = f_n

Now, we calculate $F_n$. The entire calculation of the coefficients is done like this (explained below):

In [ ]:
F_n = np.array([(f_k * exp(-2j * pi * n * k / N)).sum() for n in range(N)])

The inner part (`f_k * np.exp(... * k ...)`) uses NumPy magic. Because both `k` and `f_k` are NumPy arrays, they perform the calculation for all values of the array, without us requiring to build a loop. Multiplying the two arrays (using the `*` operator) simply performs element-wise multiplication, like this:

In [ ]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
print(a)
print(b)
print(a * b)

The outer part (`[(...) for n in range(N)]`) is a list comprehension which loops over `n`, and creates a list, which we then turn into an array using `np.array()`. Then, we have an array of all $F_n$.

Unfortunately, we do have rounding errors in our calculations, leading to slightly different results between the two algorithms. The difference between the results can be significant, but only for very, very small values of the coefficients. We're only going to print the coefficients in the neighbourhood of our frequency `f`.

In [ ]:
start, end = f - 10, f + 10
print('\tF_n\t\tDFT_list')
for n, (coefficient1, coefficient2) in enumerate(zip(F_n[start:end], DFT_list[start:end]), start):
    print("F_{:<3d} =\t{:.2e}\t{:.2e}".format(n, np.absolute(coefficient1), np.absolute(coefficient2)))

We can plot the two results to visually inspect if they are (more or less) identical:

In [ ]:
plt.plot(np.absolute(F_n))
plt.plot(np.absolute(DFT_list))

And again, the *inverse* Fourier transform as given on Wolfram Alpha:
$$
f_k = \frac{1}{N} \sum_{n=0}^{N-1} F_n e^{2\pi ikn/N}.
$$

In [ ]:
n = np.arange(N)
f2_k = 1 / N * np.array([(F_n * exp(2j * pi * k * n / N)).sum() for k in range(N)])

In [ ]:
for n, (coefficient1, coefficient2) in enumerate(zip(f_n, f2_k[:10])):
    print("f_{:<3d} = {}\t{}".format(n, coefficient1, coefficient2))
    
print("\nMaximum deviation from zero for the imaginary parts: {}j".format(np.max(np.abs(np.imag(f2_k)))))

f2_k = np.real(f2_k)

Again, rounding errors result in small imaginary values.

In [ ]:
plt.plot(f_n)
plt.plot(f2_k);

### Why bother with NumPy?

Well, for two reasons, basically. Firstly, once you get the hang of it, writing the Fourier transform using NumPy was *much* shorter than writing the function. Secondly, the NumPy code runs *much* faster. Let's time and compare the two methods:

In [ ]:
%timeit DFT(f_list)

In [ ]:
%timeit np.array([(f_k * exp(-2j * pi * n * k / N)).sum() for n in range(N)])

That's a speedup factor of about 20-30, depending on the exact signal.

## Part 2

### Generating a tone and plotting a spectrogram

In [ ]:
from scipy import signal

In [ ]:
def make_tone(frequency, samplerate, duration):
    t = np.linspace(0, duration, samplerate * duration)
    signal = np.sin(2 * pi * frequency * t)
    return t, signal

In [ ]:
SAMPLE_RATE = 44100
_, tone = make_tone(1000, SAMPLE_RATE, 2)

In [ ]:
freq, time, psd = signal.spectrogram(tone, fs=SAMPLE_RATE)

The function `pcolormesh` wants the arguments reversed or the color values transposed. I don't know why, but you can flip the $x-$ and $y-$axes and *not* transpose the power spectral density array.

In [ ]:
f = plt.pcolormesh(freq, time, psd.T)
plt.colorbar(f)
plt.xlabel("Frequency")
plt.ylabel("Time");

### Comparing two resampling methods: `resampy.resample` and `scipy.signal.resample`

In [ ]:
import resampy

NEW_SAMPLE_RATE = 16000

In [ ]:
def plot_spectrogram(s, sample_rate):
    freq, time, psd = signal.spectrogram(s, fs=sample_rate)
    f = plt.pcolormesh(freq, time, psd.T)
    plt.colorbar(f)
    plt.xlabel("Frequency")
    plt.ylabel("Time");

In [ ]:
resampled1 = resampy.resample(tone, SAMPLE_RATE, NEW_SAMPLE_RATE)
plot_spectrogram(resampled1, NEW_SAMPLE_RATE)

In [ ]:
duration = len(tone) / SAMPLE_RATE
num_samples = int(duration * NEW_SAMPLE_RATE)
resampled2 = signal.resample(tone, num=num_samples)
plot_spectrogram(resampled2, NEW_SAMPLE_RATE)

### Repeat, using a sample audio file

In [ ]:
from scipy.io import wavfile

Import the audio file and only take the first two seconds from the left audio channel. This makes it easier to visually inspect the spectrogram.

In [ ]:
sample_rate, audio = wavfile.read('muziek_48kHz.wav')
sample = audio[:2 * sample_rate, 0]

In [ ]:
plot_spectrogram(sample, sample_rate)

In [ ]:
resampled1 = resampy.resample(sample, sample_rate, NEW_SAMPLE_RATE)
plot_spectrogram(resampled1, NEW_SAMPLE_RATE)

In [ ]:
duration = len(sample) / sample_rate
num_samples = int(duration * NEW_SAMPLE_RATE)
resampled2 = signal.resample(sample, num=num_samples)
plot_spectrogram(resampled2, NEW_SAMPLE_RATE)

Both resampling methods seem to return more or less the same result. Of course, since we resample down to 16000 samples per second, we lose all frequency components above 8000 Hz. In that sense, the resampled signal is very different from the original. Still, both `resampy` and `scipy.signal` seem to do a decent job. Let's make sure the two methods actually *are* different:

In [ ]:
resampled1 == resampled2

Well, that settles it. They are numerically different, but both spectrograms look about the same.